In [1]:
# Importing all the required libraries

import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing
from tensorflow.keras import preprocessing, utils
import os
import re 
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk.data
import pickle
import speech_recognition as sr
import pyttsx3

In [2]:
# Importing data

lines = open("dataset.txt", encoding = 'utf-8', errors = 'ignore').read()

In [3]:
#Removed URLs and other unwanted objects in the text file using regular expression. 

lines = re.sub('https://\S+','', lines)
lines = re.sub('Description','', lines)
lines = re.sub('Dialogue','', lines)
lines = re.sub('Patient:','', lines)
lines = re.sub('Doctor:','', lines)
lines = re.sub('\n\n\n','', lines)
lines = re.sub(r'\Aid', ' ', lines)
lines = re.sub(r'[=]', '', lines)
lines = re.sub(r'\d', '', lines)
new_lines = lines.split('\n')
#print(new_lines)

In [5]:
# Splitting the data into question (patient) and answers (doctore).

patient = []
doctor = []

for i in range(0, len(new_lines)-1, 2): 
    patient.append(new_lines[i])
    doctor.append(new_lines[i+1])
print(len(doctor), len(patient))

1968 1968


In [6]:
# Removing empty rows
while '' in patient:
    patient.remove('')
while '' in doctor:
    doctor.remove('')

In [7]:
# Converting the list to dataframe
pat = pd.DataFrame(patient)
doc = pd.DataFrame(doctor)

In [8]:
# Converting the text to lower case 
pat = pat.astype(str)
pat[0] = pat[0].str.lower()
#print(pat)
doc = doc.astype(str)
doc[0] = doc[0].str.lower()

In [9]:
# Removing stopwords from the text

stop = stopwords.words('english')
pat[0] = pat[0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#print(pat)
doc[0] = doc[0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(pat[0])

0                                                        
1       hello doctor, get cough last days, heavy night...
2                   hello, understand concern. questions.
3       phlegm lot. tiny amount comes time. difficulty...
4       hi, would recommend take n-acetylcysteine mg p...
                              ...                        
1198    get coronavirus, ways avoid getting illnesses ...
1199    plane march th home ever since self quarantine...
1200               son fever, concerned due coronavirus ?
1201    coronavirus symptoms mild people versus severe...
1202    good day, weeks pregnant profession teaching, ...
Name: 0, Length: 1203, dtype: object


In [10]:
# Stemming 
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
pat[0] = pat[0].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
pat[0] = pat[0].str.replace('[^\w\s]','')


/var/folders/h8/4m_c3pfj1cnd2kpvtxkflc600000gn/T/ipykernel_46293/1147321814.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  pat[0] = pat[0].str.replace('[^\w\s]','')


In [11]:
doc[0] = doc[0].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
doc[0] = doc[0].str.replace('[^\w\s]','')

/var/folders/h8/4m_c3pfj1cnd2kpvtxkflc600000gn/T/ipykernel_46293/1418476608.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  doc[0] = doc[0].str.replace('[^\w\s]','')


In [12]:
# Lemmetization 
nltk.download('wordnet')
nltk.download('omw-1.4')


wordnet_lemmatizer = WordNetLemmatizer()
lemmatize_words = np.vectorize(wordnet_lemmatizer.lemmatize)
lemmatized_pat = ' '.join(lemmatize_words(pat[0]))
lemmatized_doc = ' '.join(lemmatize_words(doc[0]))

#lemmatized_pat

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kedarnandiwdekar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kedarnandiwdekar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
tokenizer = preprocessing.text.Tokenizer()
tok_pat = tokenizer.fit_on_texts(patient)
v_size = len( tokenizer.word_index )+1
print(v_size)

4103


In [14]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(doctor)
v_size = len( tokenizer.word_index )+1
print(v_size)

4019


In [15]:
# Encoder input done ! 

patient_tokens = tokenizer.texts_to_sequences(patient) # Converting all the patient data into tokens which are then 
                                                       # arranged into sequences 
patient_max = len(patient_tokens[0]) # Calculation the maximum length of longest sequence
for x in patient_tokens:
    if len(x) > patient_max:
        patient_max = len(x)
e_data = np.array(pad_sequences(patient_tokens, maxlen = patient_max, padding = 'post')) # padding the sequences and then
                                                                                         # converting them into numpy arrays
print(e_data.shape)

(1203, 292)


In [16]:
#decoder input and decoder output 

doctor_tokens = tokenizer.texts_to_sequences(doctor) # Converting all the patient data into tokens which are then
doctor_max = len(doctor_tokens[0])                   # arranged into sequences
for x in doctor_tokens:                              # Calculation the maximum length of longest sequence
    if len(x) > doctor_max:
        doctor_max = len(x)
d_data = np.array(pad_sequences( doctor_tokens, maxlen = doctor_max, padding = 'post'))
# Applying onehotencoding
dout_data = np.array(utils.to_categorical(pad_sequences( doctor_tokens, maxlen = doctor_max, padding = 'post'))) 
#decoder_input_data = np.array(padded_answers)
d_data = d_data[0:1203]         # Snipping the data to fit into the encoder
dout_data = dout_data[0:1203]   # Snipping the data to fit into the encoder
print(d_data.shape, doctor_max, dout_data.shape)



(1203, 329) 329 (1203, 329, 4019)


In [17]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.activations import softmax

e_inputs = Input(shape=(patient_max, ))                           # Creating a tensor to insert into embedder
e_embedd = Embedding( v_size, 200 , mask_zero=True)#(e_inputs) # Creating dense vector using the tensor we just fed it
e_emb = e_embedd(e_inputs)
e_lstm = LSTM( 200 , return_state=True )#(e_embedd)
e_out , e_state_h , e_state_c =  e_lstm(e_emb)# Creating encoder states which will be accessed later
e_states = [ e_state_h , e_state_c ]
 
d_inputs = Input(shape=( doctor_max ,  ))                        # Creating a tensor to insert into embedder 
d_embedd = Embedding( v_size, 200 , mask_zero=True) #(d_inputs) # Creating dense vector using the tensor we just fed it
d_emb = d_embedd(d_inputs)
d_lstm = LSTM( 200 , return_state=True , return_sequences=True ) # We created LSTM layer 
d_out , _ , _ = d_lstm ( d_emb, initial_state=e_states )   
d_dense = Dense( v_size , activation=softmax ) # The dense layer
d_output = d_dense ( d_out ) 




2023-05-14 17:13:37.186575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Model building and training. 

from tensorflow.keras import Model
chatbot_model = Model([e_inputs, d_inputs], d_output ) # Created the model to train
chatbot_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
# Compiling the model and setting up parameters for testing 
chatbot_model.summary()
chatbot_model.fit([e_data , d_data], dout_data, batch_size=50, epochs= 10)  # Function to train model


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 292)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 329)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 292, 200)     803800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 329, 200)     803800      ['input_2[0][0]']                
                                                                                              

In [19]:
from keras.models import load_model
chatbot_model.save("chatbot_model.h5", chatbot_model) # Save the trained model
model = load_model("chatbot_model.h5") # Load the trained model

In [20]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras import Model

r = sr.Recognizer() # Creating an instance for the listener, to listen to our speech 
tokens_list = list() 

def SpeakText(command):  # Initializing the listener engine 
    engine = pyttsx3.init()
    engine.say(command)
    engine.runAndWait()
    

d_state_h = Input(shape=( 200 ,))       # Initializing decoder state h
d_state_c = Input(shape=( 200 ,))       # Initializing decoder state c
d_states_in = [d_state_h, d_state_c]                    # Combining them, so as to feed it into the lstm layer
d_out, s_h, s_c = d_lstm(d_emb , initial_state=d_states_in) # lstm layer with embedded user input and initial states
d_states = [s_h, s_c]                                   # saving new states created after lstm layer
d_out = d_dense(d_out)                                  # feeding the output of the lstm into the dense layer. 
model_d = Model([d_inputs] + d_states_in,[d_out] + d_states)  # Creating the decoder model
model_e = Model(e_inputs, e_states)                     # Encoder model with initial states for user input

print("Please tell us your query.")
while (True):                                           # Creating the loop to keep running the chatbot
    try:
        # For speech to text to speech, initiating engines 
        with sr.Microphone() as source2:
            r.adjust_for_ambient_noise(source2, duration=0.2) # Adjusting for surrounding noise
            audio2 = r.listen(source2)                        # Listening to the human speech
            MyText = r.recognize_google(audio2)               # Converting the speech to text 
            MyText = MyText.lower()                           # converting the text to all lower case 
            print(MyText)
        words = MyText.lower().split()# Splitting the words in the text
        w_len = len(words)
        for i in range(0, (w_len-1)):
            w = words[i]
            tokens_list.append( tokenizer.word_index[ w ] ) # Tokeninzing the words
        user_input_padded = pad_sequences( [tokens_list] , maxlen=patient_max, padding='post') # padding the sequence
        s_values = model_e.predict(user_input_padded) # Feeding the padded sequence into the encoder model for prediction
        #s_values = model_e.predict( text_convert(MyText))
        t_seq = np.zeros( ( 1 , 1 ) )                         # Creating an matrix of 1's
        t_seq[0, 0] = tokenizer.word_index['start']           # tokenizing 'start' word at 0,0 position in the matrix
        quit_chatbot = 0                                      # initializing variable to shut the chatbot
        chatbot_answer = ''                                   # initializing the string to store chatbot response
        while quit_chatbot == 0:                              # Setting chatbot quit condition to not true 
            d_o , h , c = model_d.predict([ t_seq ] + s_values ) # Taking the decoder output with the updated states 
                                           # The inputs gone through the encoder to the decoder changing the decoder states.
                                           # the output states here are the updated deoder states
            s_index = np.argmax( d_o[0, -1, :] ) # Converting the output of the decoder into word index 
            s_word = None                        # Setting sample word to none
            for w , i in tokenizer.word_index.items() : # Iterating throughout the indexes till we get a match for the words. 
                if s_index == i :
                    chatbot_answer += ' {}'.format( w ) # converting the indexes into the matching words and formulating the response
                    s_word = w
            if s_word == 'end' or len(chatbot_answer.split()) > doctor_max: # if the formulated word is end, chabot quit condition 
                                                                            # becomes true
                quit_chatbot = 1                                            # Chabot quit condition turns true
            new_t_seq = np.zeros( ( 1 , 1 ) )            # Resetting the matrix to only 1's for the new input. 
            new_t_seq[ 0 , 0 ] = s_index
            t_seq = new_t_seq
            s_values = [ h , c ]
        print("Chatbot: " + chatbot_answer )# printing the chabot response
        SpeakText(chatbot_answer)
        #engine = pyttsx3.init()
        #engine.say(chatbot_answer)                      # Function to convert the textual bot response to speech
        #engine.runAndWait()
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
    except sr.UnknownValueError:
        print("Please speak again")                    # Error handling, in case the human speech is not recognized. 
    

Please tell us your query.
Please speak again
hello
1/1 [==============================] - 0s 27ms/step
Chatbot:  to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to t

KeyboardInterrupt: 

In [ ]:
# References that were used to build this chatbot 
# https://numpy.org/
# https://pypi.org/project/numpy/
# https://pypi.org/project/tensorflow/
# https://keras.io/api/layers/recurrent_layers/lstm/
# https://keras.io/api/models/
# A. Graves, A.-R. Mohamed, & G. Hinton. (n.d.). Speech Recognition with Deep Recurrent Neural Networks.
# Eslam, A., Ahmed , H., Omar , F., Albert , L., Youssef , M., & Michel , A. (2021). A Proposed Chatbot Framework for COVID-19. Cairo: IEEE.
# https://github.com/ShrishtiHore/Conversational_Chatbot_using_LSTM
# Wani, G. D. (2018). TEXT NORMALIZATION FOR SPEECH RECOGNITION USING DEEP LEARNING. California : California State University.
# El Hefny, W., El Bolock, A., Herbert, C., & Abdennadher, S. (2021). Chase Away the Virus: A Character-Based Chatbot for COVID-19. IEEE 2021.
# https://www.icliniq.com/qa/covid-19/i-have-cough-with-no-travel-history-is-this-a-symptoms-of-covid-19
# https://keras.io/api/losses/
# https://www.nltk.org/
# https://pypi.org/project/nltk/
# https://pythonprogramming.net/tokenizing-words-sentences-nltk-tutorial/
# https://flask.palletsprojects.com/en/2.2.x/
# https://www.tutorialspoint.com/flask/index.htm
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python
# https://www.analyticsvidhya.com/blog/2020/04/how-to-deploy-machine-learning-model-flask/
# Boyd, C. (2018). The Past Present and the Future of Speech Recognition Technology. Medium.
# Grinberg, M. (n.d.). Flask Web Development: Developing Web Applications with Python. O'Reilly.

